In [1]:
!pip install pymupdf==1.25.1 langchain==0.0.202 transformers==4.30.2 faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 408.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: tenacity
    Foun

In [ ]:
# تثبيت المكتبات المطلوبة
# pip install pymupdf langchain transformers faiss-cpu

import fitz  # PyMuPDF لاستخراج النصوص من PDF
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch

# استخراج النص من PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# تحميل النصوص من ملف PDF
pdf_text = extract_text_from_pdf("/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf")

# إنشاء تضمينات النصوص باستخدام hkunlp/instructor-large
embedding_model = HuggingFaceEmbeddings(model_name="hkunlp/instructor-large")
docstore = FAISS.from_texts([pdf_text], embedding_model)

# تحميل نموذج Llama لتوليد الإجابات
llm_pipeline = pipeline(
    "text-generation",
    model="DeepMount00/Mistral-RAG",
    device_map="auto",  # تعيين device_map إلى "auto"
    torch_dtype=torch.bfloat16  # تعيين torch_dtype إلى torch.bfloat16
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# إعداد نظام RAG
retriever = docstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # نوع السلسلة (يمكن تغييره إذا كنت تستخدم نوعًا آخر)
    retriever=retriever
)

# طرح سؤال والإجابة باستخدام النصوص المسترجعة
#"What is the content of the file?"
query = "ما هو محتوى الملف؟"
answer = qa_chain.run(query)

print("الإجابة:", answer)


In [ ]:
mistralai/Mistral-7B-Instruct-v0.3

In [13]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

'/root/mistral_models/7B-Instruct-v0.3'

In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("hkunlp/instructor-large")
model = AutoModel.from_pretrained("hkunlp/instructor-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [3]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushi

In [2]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2


In [2]:
!pip install bitsandbytes==0.41.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.2 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [3]:
!pip install bitsandbytes==0.43.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.1
    Uninstalling bitsandbytes-0.41.1:
      Successfully uninstalled bitsandbytes-0.41.1


شغال رائع

In [1]:
# تثبيت المكتبات المطلوبة
# pip install pymupdf langchain transformers faiss-cpu
!pip install bitsandbytes==0.43.2
import fitz  # PyMuPDF لاستخراج النصوص من PDF
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # Import necessary modules
import torch

# استخراج النص من PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# تحميل النصوص من ملف PDF
pdf_text = extract_text_from_pdf("/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf")

# إنشاء تضمينات النصوص باستخدام hkunlp/instructor-large
embedding_model = HuggingFaceEmbeddings(model_name="hkunlp/instructor-large")
docstore = FAISS.from_texts([pdf_text], embedding_model)

# تحميل نموذج Llama لتوليد الإجابات
# Remove load_in_8bit from here as it's not a valid argument for model.generate()
import torch
#from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # Already imported above

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")  # Specify device_map
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Redefine llm_pipeline using the loaded model and tokenizer
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",  # or device_map="auto" if using multiple GPUs
    torch_dtype=torch.bfloat16,
    # إضافة معلمات التحكم
    max_new_tokens=50,  # تحديد عدد توكن المخرجات
    do_sample=True,  # تمكين Decoding Sampling
    top_k=50,  # تحديد عدد التوكن المحتملة
    top_p=0.95,  # تحديد احتمالية اختيار توكن من بين التوكن المحتملة
    temperature=0.2,  # ضبط درجة عشوائية اختيار التوكن
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

# إعداد نظام RAG
retriever = docstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # نوع السلسلة (يمكن تغييره إذا كنت تستخدم نوعًا آخر)
    retriever=retriever
)

# طرح سؤال والإجابة باستخدام النصوص المسترجعة
# "What is the content of the file?"
query = "What is the content of the file?"
answer = qa_chain.run(query)

print("الإجابة:", answer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


الإجابة:  The content of the file is the first chapter of a book titled "Percy Jackson and the Olympians: The Lightning Thief" by Rick Riordan. The chapter is titled "Look, I Didn't Want to Be a


شغااااال جيد

In [2]:
# تثبيت المكتبات المطلوبة
# pip install pymupdf langchain transformers faiss-cpu

import fitz  # PyMuPDF لاستخراج النصوص من PDF
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch

# استخراج النص من PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# تحميل النصوص من ملف PDF
pdf_text = extract_text_from_pdf("/content/sample-5.pdf")

# إنشاء تضمينات النصوص باستخدام hkunlp/instructor-large
embedding_model = HuggingFaceEmbeddings(model_name="hkunlp/instructor-large")
docstore = FAISS.from_texts([pdf_text], embedding_model)

# تحميل نموذج Llama لتوليد الإجابات
llm_pipeline = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.3",
    device_map="auto",  # تعيين device_map إلى "auto"
    torch_dtype=torch.bfloat16  # تعيين torch_dtype إلى torch.bfloat16
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# إعداد نظام RAG
retriever = docstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # نوع السلسلة (يمكن تغييره إذا كنت تستخدم نوعًا آخر)
    retriever=retriever
)

# طرح سؤال والإجابة باستخدام النصوص المسترجعة
#"What is the content of the file?"
query = "What is the content of the file?"
answer = qa_chain.run(query)

print("الإجابة:", answer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


الإجابة:  The content of the file is a resume, including the applicant's name, contact information,
